## Séparer notre ensemble de Question-Réponses
Dans le but d'entrainer plusieurs fois notre modèle sur plusieurs ensemble d'entrainement possible, nous avons créé ce notebook (section) qui explique nos motivations et la façon dont nous avons obtenu tous nos ensembles d'entrainement et de tests. 

Les résultats de chaque expérience/extraction de sets seront sauvegardés dans le fichier ```../datasets/ready-to-train```

In [40]:
# imports
import numpy as np
import pandas as pd

# Constantes
# PATH = "../datasets/Nos datasets"
PATH = "./"
FILE_NAME = "/Data collection - Data_FR.tsv"
SEP = "\t"
COL_NAMES = ["Question", "Reponse", "Catégorie", "Type", "Source", "FR?"]

# lecture et affichage
df_to_split = pd.read_csv(PATH+FILE_NAME, sep=SEP, usecols=COL_NAMES)
df_to_split.drop(labels=0, axis=0, inplace=True)

print("Nous avons :", len(df_to_split), "couples Q&A.")
df_to_split.head(5)

Nous avons : 51449 couples Q&A.


,Question,Reponse,Catégorie,Type,Source,FR?
1,Quels sont tes centres d'interets,Je suis interessé par beaucoup de choses. Nous...,BotProfile,QR,chatterbot-corpus-french,Oui
2,Quels sont tes centres d'interets,Je suis interessé par une grande variete de su...,BotProfile,QR,chatterbot-corpus-french,Oui
3,Quels sont tes sujets preferes,"Mes sujets preferes incluent la robotique, l'i...",BotProfile,QR,chatterbot-corpus-french,Oui
4,Quel est ton numero,Je n'ai pas de numero.,BotProfile,QR,chatterbot-corpus-french,Oui
5,Quel est ton numero,"5446, c'est mon numero maintenant.",BotProfile,QR,chatterbot-corpus-french,Oui


### Sans conditions sur les sous-ensembles dont se compose le dataset
Notre ensemble étant conçus de plusieurs dialogues, Q&A... d'autres dataset (traduit ou non ; détails dans le fichier ```TODO: À METTRE PLUS TARD```), le but ici est de créer un test set avec K couples de Q&A tirés de façon totalement aléatoire. 

Le test set sera unique quand aux questions, aucune vérifications sur les réponses. Autrement dit, *nous n'auraons pas des questions identiques mais un doublon de réponse pourrait apparaître.*

In [41]:
from sklearn.model_selection import train_test_split

# Avoir 1000 données pour l'ensemble de test
train_df, test_df = train_test_split(df_to_split, shuffle=True, test_size=1000*1.0/len(df_to_split), random_state=21)
train_df.shape, test_df.shape



((50449, 6), (1000, 6))

In [42]:
# Avoir 1000 données pour l'ensemble de test
train_df2, test_df2 = train_test_split(df_to_split, shuffle=True, test_size=5000*1.0/len(df_to_split), random_state=21)
train_df2.shape, test_df2.shape



((46449, 6), (5000, 6))

In [43]:
# Get names of indexes for which the question is the same in the training set as in the test set
indexNames = train_df[ train_df['Question'].isin(list(test_df['Question']) )].index

# mask sur les questions existantes sur le test
rows_to_remove = train_df.loc[indexNames]
print("Nous allons devoir supprimer ", len(rows_to_remove), "questions du train_set car ces dernières apparaîssent dans le test_set.\n")

# Delete these row indexes from dataFrame
train_df.drop(indexNames , inplace=True)

OUTPUT_PATH = "./output/"
train_df.to_csv(OUTPUT_PATH + "train_df_1000.tsv", sep="\t", index=False)
test_df.to_csv(OUTPUT_PATH + "test_df_1000.tsv", sep="\t", index=False)

# The size of the training set goes down a bit
print("Respectivement, la taille du train et test set:")
train_df.shape, test_df.shape

Nous allons devoir supprimer  916 questions du train_set car ces dernières apparaîssent dans le test_set.

Respectivement, la taille du train et test set:


((49533, 6), (1000, 6))

In [44]:
# Get names of indexes for which the question is the same in the training set as in the test set
indexNames2 = train_df2[ train_df2['Question'].isin(list(test_df2['Question']) )].index

# mask sur les questions existantes sur le test
rows_to_remove2 = train_df2.loc[indexNames2]
print("Nous allons devoir supprimer ", len(rows_to_remove2), "questions du train_set car ces dernières apparaîssent dans le test_set.\n")

# Delete these row indexes from dataFrame
train_df2.drop(indexNames2 , inplace=True)

OUTPUT_PATH = "./output/"
train_df2.to_csv(OUTPUT_PATH + "train_df2_5000.tsv", sep="\t", index=False)
test_df2.to_csv(OUTPUT_PATH + "test_df2_5000.tsv", sep="\t", index=False)

# The size of the training set goes down a bit
print("Respectivement, la taille du train et test set:")
train_df2.shape, test_df2.shape


Nous allons devoir supprimer  3611 questions du train_set car ces dernières apparaîssent dans le test_set.

Respectivement, la taille du train et test set:


((42838, 6), (5000, 6))

### Avec conditions sur les sous-ensembles dont se compose le dataset
Notre dataset actuel/final est composé de plusieurs sous-datasets, nous aimerons en masquer quelques un (c'est-à-dire ne pas les inclure dans le train) pour voir quel comportement pourrait avoir le ChatBot.

In [45]:
df_to_split["Catégorie"].value_counts()

BotProfile       9817
Conversations    5660
Psy               781
Divers            295
Emotion           207
Psycho             65
Nourriture          6
XXXX                1
Name: Catégorie, dtype: int64

In [46]:
df_to_split["Source"].value_counts()

Facebook-Research            24377
chatbot-professional          9797
Circa                         9614
DialogueJarvis                3725
DialogueQA                    1128
CounsilChat                    781
DialogueHumanChat              760
Elizia                         617
chatterbot-corpus-english      527
chatbot-depression              65
chatterbot-corpus-french        48
freestyle                       10
Name: Source, dtype: int64

#### 1. En masquant le 

In [47]:
mask_fb_research = (df_to_split["Source"] == "Facebook-Research")
df_wo_fbresearch = df_to_split[~mask_fb_research]

df_wo_fbresearch = df_wo_fbresearch[["Question", "Reponse"]]

df_wo_fbresearch.to_csv(OUTPUT_PATH + "wo_fb_research.tsv", sep="\t", index=False)

In [48]:
mask_professional = (df_to_split["Source"] == "chatbot-professional")
mask_circa = (df_to_split["Source"] == "Circa")

mask =  mask_circa + mask_professional # 'or' 

df_wo_professional_and_circa = df_to_split[~mask]

df_wo_professional_and_circa = df_wo_professional_and_circa[["Question", "Reponse"]]

df_wo_professional_and_circa.to_csv(OUTPUT_PATH + "df_wo_professional_and_circa.tsv", sep="\t", index=False)

In [51]:
!zip -r /content/output.zip /content/output/

  adding: content/output/ (stored 0%)
  adding: content/output/train_df_1000.tsv (deflated 70%)
  adding: content/output/test_df2_5000.tsv (deflated 69%)
  adding: content/output/wo_fb_research.tsv (deflated 74%)
  adding: content/output/train_df2_5000.tsv (deflated 70%)
  adding: content/output/test_df_1000.tsv (deflated 69%)
  adding: content/output/df_wo_professional_and_circa.tsv (deflated 67%)


In [52]:
from google.colab import files
files.download("/content/output.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>